# Прогнозирование цен потребительского ритейла по тестовой выборке на основе глубоких нейронных сетей
__Выполнил:__ *Домченко Максим*

__Студент группы:__ *РИМ-130962*

In [ ]:
# Для локального запуска загрузите kaggle.json в ~/.kaggle/kaggle.json и дайте права:
# !chmod 600 ~/.kaggle/kaggle.json

In [16]:
# Для запуска в colab

# from google.colab import files
# uploaded = files.upload('/root/.kaggle')
# !chmod 600 /root/.kaggle/kaggle.json

In [37]:
# 1. Скачиваем данные
!kaggle competitions download -p ../data -c favorita-grocery-sales-forecasting --force

 57%|██████████████████████▉                 | 262M/458M [00:00<00:00, 2.74GB/s]
100%|████████████████████████████████████████| 458M/458M [00:00<00:00, 2.80GB/s]


In [38]:
# 2. Разархивируем
import zipfile
import os
import py7zr

# === Настройки ===
zip_path = "../data/favorita-grocery-sales-forecasting.zip"        # Укажи путь к ZIP
extract_dir = "../data"        # Папка для всех извлечений

# === Шаг 1: Распаковать ZIP ===
os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# === Шаг 2: Найти и распаковать все .7z архивы ===
for root, dirs, files in os.walk(extract_dir):
    for file in files:
        if file.endswith(".7z"):
            file_path = os.path.join(root, file)
            with py7zr.SevenZipFile(file_path, mode='r') as archive:
                archive.extractall(path=extract_dir)
            os.remove(file_path)  # Удаляем .7z после распаковки

# === Шаг 3: Удалить исходный ZIP ===
os.remove(zip_path)

print("✅ Готово: всё распаковано и исходники удалены.")

✅ Готово: всё распаковано и исходники удалены.
